In [1]:
#conda install pytorch torchvision torchaudio cudatoolkit=10.2 -c pytorch

import numpy as np
import pandas as pd

import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')

Using cuda device


In [2]:
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

#score results
def score_model(true, pred, columns = ""):
    cm = confusion_matrix(true,pred)
    l = np.sum(cm)
    cm = cm/l
    s = [accuracy_score(true, pred), matthews_corrcoef(true, pred), f1_score(true,pred), cm[0,0], cm[1,1],cm[0,1],cm[1,0]]
    return pd.DataFrame(data = s, index = ['accuracy', 'matthew_corr', 'f1', 'tn', 'tp','fp','fn'], columns = [columns])


In [22]:
#prediction
class LSTM(nn.Module):
    def __init__(self, num_layers=1, dropout = 0):
        super(LSTM, self).__init__()
        self.lstm = nn.LSTM(768+1,1, num_layers, dropout = dropout)
    def forward(self, x):
        output, hidden = self.lstm(x)
        output = torch.squeeze(output)
        return output
    
#classification
class LSTM2(nn.Module):
    def __init__(self, num_layers=1, dropout = 0):
        super(LSTM2, self).__init__()
        self.lstm = nn.LSTM(768+1,1, num_layers, dropout = dropout)
        self.trading_strategy = nn.Sigmoid()
    def forward(self, x):
        output, hidden = self.lstm(x)
        output = torch.squeeze(output)
        output = self.trading_strategy(output)
        return torch.squeeze(torch.tensor_split(output,[4], dim = 1)[1])   

In [4]:
#import btc price history
btc_prices = pd.read_csv("btc_prices.csv", index_col = 0)
btc_prices.index = pd.to_datetime(btc_prices.index).strftime('%Y-%m-%d')
btc_prices['log_ret'] = np.log(btc_prices.Close/btc_prices.Close.shift(1))
btc_ret = btc_prices[["log_ret"]].dropna()

#get the right daterange
btc_ret_train = btc_ret[(btc_ret.index >= '2016-01-01') & (btc_ret.index < '2021-11-01')].sort_index()

topic_vectors = pd.read_csv("bitcoin/btc_2016.csv", index_col = 0)
for i in [2017,2018,2019,2020, 2021]:
    topic_vectors = topic_vectors.append(pd.read_csv(f"bitcoin/btc_{i}.csv", index_col = 0))
topic_vectors.index = pd.date_range('2016-01-01', periods=len(topic_vectors)).strftime('%Y-%m-%d')
topic_vectors = btc_ret_train.join(topic_vectors, how= "left").dropna()

In [4]:
#import eth price history
eth_prices = pd.read_csv("eth_prices.csv", index_col = 0)
eth_prices.index = pd.to_datetime(eth_prices.index).strftime('%Y-%m-%d')
eth_prices['log_ret'] = np.log(eth_prices.Close/eth_prices.Close.shift(1))
eth_ret = eth_prices[["log_ret"]].dropna()

#get the right daterange
eth_ret_train = eth_ret[(eth_ret.index >= '2014-01-01') & (eth_ret.index < '2021-11-01')].sort_index()

topic_vectors = pd.read_csv("bitcoin/eth_2014.csv", index_col = 0)
for i in [2015, 2016, 2017,2018,2019,2020, 2021]:
    topic_vectors = topic_vectors.append(pd.read_csv(f"bitcoin/eth_{i}.csv", index_col = 0))
topic_vectors.index = pd.date_range('2014-01-01', periods=len(topic_vectors)).strftime('%Y-%m-%d')
topic_vectors = eth_ret_train.join(topic_vectors, how= "left").dropna()

In [5]:
#get the time series nicely streamlined into the machine learning models
rolling_window = 5

X_data = np.empty([len(topic_vectors)-rolling_window,rolling_window,769])
y_series_data = np.empty([len(topic_vectors)-rolling_window,rolling_window])
for i in range(0,len(topic_vectors)-rolling_window):
    X_data[i] = np.array(topic_vectors.iloc[i:i+rolling_window,:])
    y_series_data[i] = np.array(topic_vectors.iloc[i+1:i+1+rolling_window,0])
y_data = np.array(topic_vectors.iloc[rolling_window:,0])

train_cutoff = -3000
tune_cutoff = -604
test_cutoff = -304

X = X_data[:tune_cutoff]
X_tune = X_data[tune_cutoff:test_cutoff]
X_test = X_data[test_cutoff:]


y = y_data[:tune_cutoff]
y_tune = y_data[tune_cutoff:test_cutoff]
y_test = y_data[test_cutoff:]

y_series = y_series_data[:tune_cutoff]
y_series_test = y_series_data[test_cutoff:]

In [10]:
def train_loop(dataloader, model, loss_fn, optimizer, new = True):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    loss, current = loss.item(), batch * len(X)
    print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [8]:
#lstm
train_dataset = TensorDataset(torch.tensor(X).float(), torch.tensor(y_series).float())
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
#test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)

In [25]:
#weighted BCELoss
def BCELoss_weighted(weights):
    def loss(pred, target):
        pred = torch.clamp(pred,min=1e-7,max=1-1e-7)
        bce = - weights[1] * target * torch.log(pred) - (1 - target) * weights[0] * torch.log(1 - pred)
        return torch.mean(bce)
    return loss

In [ ]:
#hyperparameters
lr = [1e-3, 1e-2, 1e-1, 1e-4]

#find best model
for rate in lr:
    model = LSTM(1,0)

    learning_rate = rate
    batch_size = 64
    epochs = 500

    loss_fn = nn.MSELoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

    print("Rate: ",rate)
    for t in range(epochs):
        print(f"Epoch {t+1}\n-------------------------------")
        train_loop(train_dataloader, model, loss_fn, optimizer)
    x_test = torch.tensor(X_tune).float()
    pred = torch.squeeze(torch.tensor_split(model(x_test),[4], dim = 1)[1]).detach().numpy()
    print(score_model(pred>0, y_tune>0))

In [ ]:
#lstm score, sequence output, prediction
x_test = torch.tensor(X_test).float()
pred = torch.squeeze(torch.tensor_split(model(x_test),[4], dim = 1)[1]).detach().numpy()
score_model(pred>0, y_test>0)

In [13]:
#lstm classification
train_dataset = TensorDataset(torch.tensor(X).float(), torch.tensor(y>0).float())
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
#test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)

In [ ]:
#weighted BCELoss
def BCELoss_weighted(weights):
    def loss(pred, target):
        pred = torch.clamp(pred,min=1e-7,max=1-1e-7)
        bce = - weights[1] * target * torch.log(pred) - (1 - target) * weights[0] * torch.log(1 - pred)
        return torch.mean(bce)
    return loss

In [ ]:
#hyperparameters
lr = [1e-3, 1e-2, 1e-1, 1e-4]

#find best model
for rate in lr:
    model = LSTM2(1,0)

    learning_rate = rate
    batch_size = 64
    epochs = 1

    loss_fn = BCELoss_weighted(weights = [1,1])
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

    print("Rate: ",rate)
    for t in range(epochs):
        print(f"Epoch {t+1}\n-------------------------------")
        train_loop(train_dataloader, model, loss_fn, optimizer)
        x_test = torch.tensor(X_tune).float()
        pred = model(x_test).detach().numpy()
        print(score_model(pred>0.5, y_tune>0))

In [ ]:
#score, single output, classification
x_test = torch.tensor(X_test).float()
pred = model(x_test).detach().numpy()
score_model(pred>0.5, y_test>0)